In [19]:
!git clone https://github.com/Priyanshu-hawk/pytorch_random.git
!pip install torchinfo

In [4]:
%cd pytorch_random

/content/pytorch_random


In [9]:
import requests
with open("./Modular_DeepLearning/food101_3_class/data.zip", "wb") as f:
  r = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip")
  f.write(r.content)

import zipfile, os
with zipfile.ZipFile("./Modular_DeepLearning/food101_3_class/data.zip", "r") as zip_ref:
    os.makedirs("/Modular_DeepLearning/food101_3_class/data", exist_ok=True)
    zip_ref.extractall("./Modular_DeepLearning/food101_3_class/data")

In [10]:
import os
data = "./Modular_DeepLearning/food101_3_class/data"
train_path = os.path.join(data,"train")
test_path = os.path.join(data,"test")

In [11]:
from Modular_DeepLearning.food101_3_class.data_setup import createDataloader
import torchvision, torch

# model_weights = torchvision.models.ResNet101_Weights.DEFAULT
# _model = torchvision.models.resnet101(pretrained=model_weights)

model_weights = torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
_model = torchvision.models.efficientnet_b0(weights=model_weights)

auto_transform = model_weights.transforms()

auto_transform

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 83.8MB/s]


ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [12]:
train_load, test_load, class_names = createDataloader(train_path, test_path, BTACH_SIZE=32, train_transform=auto_transform, test_transform=auto_transform)

In [13]:
train_load, test_load, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7c17655412d0>,
 ['pizza', 'steak', 'sushi'])

In [14]:
# pre tranin model location
# pytoch domain lib
# timm or torch image models
# hugging face
# papers with code

In [15]:
_model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [20]:
from torchinfo import summary
summary(_model, input_size=(32,3,224,224))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [32, 3]                   --
├─Sequential: 1-1                                       [32, 1280, 7, 7]          --
│    └─Conv2dNormActivation: 2-1                        [32, 32, 112, 112]        --
│    │    └─Conv2d: 3-1                                 [32, 32, 112, 112]        864
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        64
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        --
│    └─Sequential: 2-2                                  [32, 16, 112, 112]        --
│    │    └─MBConv: 3-4                                 [32, 16, 112, 112]        1,448
│    └─Sequential: 2-3                                  [32, 24, 56, 56]          --
│    │    └─MBConv: 3-5                                 [32, 24, 56, 56]          6,004
│    │    └─MBConv: 3-6                              

In [21]:
_model.avgpool

AdaptiveAvgPool2d(output_size=1)

In [22]:
_model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

In [23]:
_model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=False),
    torch.nn.Linear(in_features=1280, out_features=3, bias=True)

).to(device="cuda")

In [24]:
_model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

In [25]:
from torchinfo import summary
summary(_model, input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [26]:
#pause training of features
for params in _model.features.parameters():
    params.requires_grad = False

In [27]:
from torchinfo import summary
summary(_model, input_size=(32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [28]:
_model = _model.to(device="cuda")

In [29]:
next(_model.parameters()).device

device(type='cuda', index=0)

In [30]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(_model.parameters(), lr=0.001)

In [31]:
from Modular_DeepLearning.food101_3_class.train import trainit

trainit(_model, train_load, test_load, loss_fn, optimizer, epochs=5, cala_test_loss_acc=True, device="cuda")

  0%|          | 0/5 [00:00<?, ?it/s]

Train: Loss - 1.0212297216057777 | Accuracy - 0.41015625


 20%|██        | 1/5 [00:04<00:18,  4.74s/it]

Test: Loss - 0.8267017602920532 | Accuracy - 0.772727370262146
Train: Loss - 0.8134996443986893 | Accuracy - 0.734375


 40%|████      | 2/5 [00:08<00:12,  4.15s/it]

...

Test: Loss - 0.6552736759185791 | Accuracy - 0.7831439971923828
Train: Loss - 0.7110308855772018 | Accuracy - 0.7421875


 80%|████████  | 4/5 [00:14<00:03,  3.24s/it]

Test: Loss - 0.6019369959831238 | Accuracy - 0.8655303716659546
Train: Loss - 0.5367121919989586 | Accuracy - 0.9140625


100%|██████████| 5/5 [00:17<00:00,  3.56s/it]

Test: Loss - 0.5333881378173828 | Accuracy - 0.866477370262146


In [35]:
from PIL import Image
from torchvision import transforms

## hardcoding the class names
# class_names = ['pizza', 'steak', 'sushi']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open("pizza.jpg", "wb") as f:
    img = requests.get("https://veenaazmanov.com/wp-content/uploads/2020/07/Mushroom-Pizza-Recipe4.jpg")
    f.write(img.content)

img_path = "pizza.jpg"
# img_path = "./steak.jpg"
img = Image.open(img_path)
img = auto_transform(img).unsqueeze(0)

img = img.to(device)

inference = _model(img)
print(inference)
print(class_names[torch.argmax(inference).item()])


tensor([[ 0.6448, -0.1007, -0.8760]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
pizza
